In [15]:
using Random, Statistics, Parameters, StatsBase, Distributions, Optim, ForwardDiff, Calculus, LinearAlgebra, DataFrames 

# Question 1

* $M = 250$ markets 
* Market characteristics: $X_{m} \sim N(3,1)$, $m = 1,2,..., 250$.
* Firm-specific characteristics: $Z_{fm} \sim N(0,1)$, $f = 1,,.,F_{m}, \quad m = 1,..,250$.



## Data generating

In [64]:
@with_kw mutable struct parameters
    M::Int64 = 250
    α::Float64 = 1.0
    β::Float64  = 1.0
end


param = parameters()
δ = 1.0;
μ = 2.0;
σ = 1.0;
tru_param = [μ,σ,δ];



## Market characteristics and draw numbers of potential entrants in each market

In [65]:
X = rand(Normal(3,1), param.M)
F = [2,3,4]
entrant = sample(MersenneTwister(342) ,F, param.M; replace = true, ordered = false);


## Draw firm-specific and unobservable fixed cost

In [67]:
uf_num = rand(MersenneTwister(123), Normal(tru_param[1], tru_param[2]), sum(entrant, dims = 1)[1])
z_firm = rand(Normal(0,1), sum(entrant, dims = 1)[1])
u_firm_new = Vector{Float64}[]
z_firm_new = Vector{Float64}[]
k = 1
j = 0
for i in eachindex(entrant)
    j += entrant[i]
    temp_1 = uf_num[k:j]
    temp_2 = z_firm[k:j]
    u_firm_new = push!(u_firm_new, temp_1)
    z_firm_new = push!(z_firm_new, temp_2)
    k = j + 1
end
Z = copy(z_firm_new);

## Get equilibrium entered firm numbers and firm's entry decisions. 

In [11]:
function eq_firm_calc(tru_param::AbstractVector, other_param::parameters, market::AbstractVector, potential::AbstractVector, Z::AbstractVector, U::AbstractVector)
    """ equilbrium entered firm calculation
    Input:
    1. tru_param::AbstractVector : true parameters - [μ, σ, δ] 
    2. other_param::parameters : other true parameters [α, β, M]
    3. Market::AbstractVector - market observables
    4. potential::AbstractVector - potential entrants for each market (M vector)
    5. Z::AbstractVector - firm observable costs
    6. U::AbstractVector - firm fixed cost unobservables 
    Output: 
    1. eq_entered::AbstractVector : equilbrium entered firm number
    2. decision_firm::AbstractVector : equilibrium firm entry decisions. 
    """
    # Initialization
    ## equilbrium firm entry : Most profitably firms enter firm
    #Profit : Π = X*β - (Z * α + u_firm)   
    eq_entered = zeros(eltype(Int64),length(potential))
    Π = similar(Z)
    rank_firm = Vector{Int64}[]
    decision_firm = Vector{Int64}[]
    for m in eachindex(potential)
        x = market[m]
        entr_num = potential[m]
        Z_m = Z[m]
        U_f = U[m]
        Π_m = zeros(eltype(Float64),entr_num)
        Π_m = x * other_param.β .- Z_m * other_param.α - U_f
        Π_m_ranked =  sort(Π_m, rev= true)
        eq_entered[m] = 0
        entrant_number = Vector(1:1:potential[m])

        Profit = Π_m_ranked - tru_param[3] * log.(entrant_number)
        eq_entered[m] = count(i -> (i>=0), Profit)


        temp1 = Profit + tru_param[3] * log.(entrant_number)
        temp2 = round.(Π_m; digits = 5)
        temp3 = round.(temp1; digits = 5)
        temp4 = zeros(eltype(Int64), potential[m])
        for j in 1: potential[m]
            temp4[j] = findall(temp2 .== temp3[j])[1]
        end
        
        rank_firm = push!(rank_firm, temp4)
        temp_d = temp4 .<= eq_entered[m]
        decision_firm = push!(decision_firm, temp_d)

        
    end
    return eq_entered , decision_firm #250 X 1 vector, firm specific cost
end


eq_firm_calc (generic function with 1 method)

In [68]:
entered_firm, decision = eq_firm_calc(tru_param, param, X, entrant, Z, u_firm_new);

## For the expositional purpose, I here create dataframe (In the actual estimation, this dataframe is not used)

In [47]:
first(data_1, 10)

,market_index,observed_profit,potential_firm_number,entry_decision,eq_firm_number
,Int64,Float64,Int64,Int64,Int64
1,1,0.914035,4,0,0
2,1,0.417636,4,0,0
3,1,-0.479137,4,0,0
4,1,-2.34252,4,0,0
5,2,2.63331,2,1,1
6,2,0.210277,2,0,1
7,3,2.17458,4,0,0
8,3,1.49642,4,0,0
9,3,1.04001,4,0,0


# Question 2 : Probit estimator

Following one of the special cases explained in Berry (1992), I focus on the probabilities of the number of firms in each market with three cases.
* $Pr(N=0)$
* $Pr(N=1)$
* $Pr(N=2)$

In [56]:

function entry_probit(param1::AbstractVector, fixed_param::parameters, market::AbstractVector, firm_char::AbstractVector, potential::AbstractVector, eq_firm::AbstractVector)
    """ loglike function
    Input:
    1. param1::AbstractVector : parameters of interest [μ, σ, δ] 
    2. fixed_param::parameters : other parameters [α, β, M]
    3. market::AbstractVector - marketwide observables : X (M vector)
    4. firm_char::AbstractVector - firm observable characteristics : 'M X entrant[m]' m=1,...,M vector
    5. potential::AbstractVector - potential entrants for each market (M vector)
    Output: 
    1. -loglik::Float64 : negative loglik value 
    """
    if param1[2] <0
        param1[2] = 1
    end

    Pr_0 = zeros(Float64, fixed_param.M) #Pr(N=0)
    Pr_1 = zeros(Float64, fixed_param.M) #Pr(N=1)
    Pr_2 = zeros(Float64, fixed_param.M) #Pr(N>=2)
    dis = Normal(param1[1],param1[2])

    for m in eachindex(potential) # Market m case
        x = market[m]
        Z_m = firm_char[m]
        entr_num = potential[m]
        ## each firm's profit 
        Π_m = zeros(eltype(Float64),entr_num)
        Π_m = x * fixed_param.β .- Z_m * fixed_param.α
        # order firms by profitability
        sort!(Π_m, rev = true)
        # Pr_1 = The first profitable firm enters so the rest firms must have negative profits
        Pr_0[m] = 1
        for i in 1: entr_num
            Pr_0[m] *= (1-cdf(dis,Π_m[i]))
        end 
        
        
        Pr_1[m] = cdf(dis, Π_m[1])*(1- cdf(dis, Π_m[2] - param1[3])) - (cdf(dis,Π_m[1]) - cdf(dis, Π_m[1] - param1[3])) * (cdf(dis,Π_m[2]) - cdf(dis,Π_m[2]-param1[3])) * (cdf(dis, Π_m[1]- Π_m[2])/2)
        Pr_2[m] = 1 - Pr_0[m] - Pr_1[m]
        
    end


    nofirm = eq_firm .== 0
    monopoly = eq_firm .== 1
    moretwo = eq_firm .>= 2
    Pr_0[Pr_0 .<= 0.0] .= 1e-10
    Pr_1[Pr_1 .<= 0.0] .= 1e-10
    Pr_2[Pr_2 .<= 0.0] .= 1e-10
        
    
    loglik = sum(nofirm .* log.(Pr_0) .+ monopoly.* log.(Pr_1) .+ moretwo .* log.(Pr_2))

    return -loglik
end



entry_probit (generic function with 1 method)

## Compute probit estimates and standard errors for ($\mu$, $\sigma$, $\delta$)
Here I use BFGS for the probit estimator and compute standard error using the information matrix.

* Do you need to make any equilibrium selection assumptions?

I made an equilibrium selection assumption that profitable firms enter first sequentially.

In [69]:
opt_probit = Optim.optimize(vars -> entry_probit(vars, param, X, Z, entrant, entered_firm), ones(3), BFGS(), Optim.Options(show_trace = false, g_tol = 1e-7));
estimates_probit = opt_probit.minimizer

3-element Vector{Float64}:
 2.167048317102835
 0.9586206048170339
 0.6026102979703265

In [58]:
hessian_probit = hessian( vars -> entry_probit(vars, param, X, Z, entrant, entered_firm)  )
se_probit = diag(inv(hessian_probit(estimates_probit)))



3-element Vector{Float64}:
 0.019013639206240877
 0.019354139457049585
 0.03598819093003883

#  Question 3 and 4 : Method of Simulated Moments

## MSM estimator (including incorrect specification cases)

In [82]:

function simulated_mm(param1::AbstractVector, param2::parameters, market::AbstractVector, firm_char::AbstractVector, eq_firm::AbstractVector, eq_firm_vec::AbstractVector, potential::AbstractVector, S::Int64, mode)
    """
    Input:
    1. param1::AbstractVector : parameters of interest : μ, σ, δ
    2. fixed_param::parameters : other parameters [α, β, M]
    3. market::AbstractVector - marketwide observables
    4. firm_char::AbstractVector - firm observable characteristics : 'M X entrant[m]' m=1,...,M vector
    5. eq_firm::AbstractVector - equilbrium entered firm number for each market 'M' 
    6. potential::AbstractVector - potential entrants for each market
    7. S::Int64 - simulation number

    Output:
    1. Criterion function value (N* - N_simulated)' * (N* - N_simulated)
    """
    if param1[2] < 0 
        param1[2] = 1.0
    end

    enter_firm = zeros(length(potential)*S)
    Z_m_temp = copy(firm_char)
    Z_m = repeat(Z_m_temp, S)
    X_m_temp = copy(market)
    X_m = repeat(X_m_temp, S)
    firm_number = repeat(potential, S)
    simu = rand(MersenneTwister(123), Normal(param1[1], param1[2]), sum(firm_number, dims = 1)[1])
    k = 1
    u_firm = Vector{Float64}[]
    j = 0
    eq_entered = repeat(eq_firm, S)
    for i in eachindex(firm_number)
        j += firm_number[i]
        temp = simu[k:j]
        u_firm = push!(u_firm, temp)
        k = j + 1
    end     
    
    if mode == "number"
        for j in eachindex(firm_number)
            Pi = param2.β * X_m[j] .- param2.α * Z_m[j] .- u_firm[j]
            sort!(Pi, rev= true)
            entrant_number = Vector(1:1:firm_number[j])
            Profit = Pi - param1[3] * log.(entrant_number)
            enter_firm[j] = count(i -> (i>=0), Profit)
        end

        proj_temp = reshape(enter_firm, param2.M, S)
        proj = sum(proj_temp, dims = 2) / S
        Q = (eq_firm - proj)' * (eq_firm - proj)
        return Q[1] 
    elseif mode =="numberrev"

        for j in eachindex(firm_number)
            Pi = param2.β * X_m[j] .- param2.α * Z_m[j] .- u_firm[j]
            sort!(Pi, rev= false)
            entrant_number = Vector(1:1:firm_number[j])
            Profit = Pi - param1[3] * log.(entrant_number)
            enter_firm[j] = count(i -> (i>=0), Profit)
        end

        proj_temp = reshape(enter_firm, param2.M, S)
        proj = sum(proj_temp, dims = 2) / S
        Q = (eq_firm - proj)' * (eq_firm - proj)
        return Q[1] 

    elseif mode == "identity"
        decision_firm = Vector{Int64}[]
        for j in eachindex(firm_number)
            Pi = param2.β * X_m[j] .- param2.α * Z_m[j] .- u_firm[j]
            Pi_ranked = sort(Pi, rev = true)
            entrant_number = Vector(1:1:firm_number[j])
            Profit = Pi_ranked - param1[3] * log.(entrant_number)
            enter_firm[j] = count(i -> (i >= 0), Profit)

            temp1 = Profit + param1[3] * log.(entrant_number)
            temp2 = round.(Pi; digits = 5)
            temp3 = round.(temp1; digits = 5)
            temp4 = zeros(eltype(Int64), firm_number[j])
            for m in 1: firm_number[j]
                temp4[m] = findall(temp2 .== temp3[m])[1]
            end
     
            
            temp_d = temp4 .<= eq_entered[j]
            decision_firm = push!(decision_firm, temp_d)
        end

        d = Vector{Int64}(undef,1)
        for m in eachindex(firm_number)
            append!(d, decision_firm[m])
        end
        d = d[2:end]
        d_temp = reshape(d, sum(potential), S)
        d_eq = Vector{Float64}(undef,1)
        for m in eachindex(potential)
            append!(d_eq, eq_firm_vec[m])
        end
        d_eq = d_eq[2:end]

        d_proj = sum(d_temp, dims = 2) ./ S

        proj_2 = reshape(enter_firm, param2.M, S)
        proj_num = sum(proj_2, dims = 2) / S
        moment = vcat((eq_firm - proj_num), (d_eq - d_proj)) 

        Q = moment' * moment

        return Q[1]

    elseif mode == "identityrev"
        decision_firm = Vector{Int64}[]
        for j in eachindex(firm_number)
            Pi = param2.β * X_m[j] .- param2.α * Z_m[j] .- u_firm[j]
            Pi_ranked = sort(Pi, rev = false)
            entrant_number = Vector(1:1:firm_number[j])
            Profit = Pi_ranked - param1[3] * log.(entrant_number)
            enter_firm[j] = count(i -> (i >= 0), Profit)

            temp1 = Profit + param1[3] * log.(entrant_number)
            temp2 = round.(Pi; digits = 5)
            temp3 = round.(temp1; digits = 5)
            temp4 = zeros(eltype(Int64), firm_number[j])
            for m in 1: firm_number[j]
                temp4[m] = findall(temp2 .== temp3[m])[1]
            end
     
            
            temp_d = temp4 .<= eq_entered[j]
            decision_firm = push!(decision_firm, temp_d)
        end

        d = Vector{Int64}(undef,1)
        for m in eachindex(firm_number)
            append!(d, decision_firm[m])
        end
        d = d[2:end]
        d_temp = reshape(d, sum(potential), S)
        d_eq = Vector{Float64}(undef,1)
        for m in eachindex(potential)
            append!(d_eq, eq_firm_vec[m])
        end
        d_eq = d_eq[2:end]

        d_proj = sum(d_temp, dims = 2) ./ S

        proj_2 = reshape(enter_firm, param2.M, S)
        proj_num = sum(proj_2, dims = 2) / S
        moment = vcat((eq_firm - proj_num), (d_eq - d_proj)) 

        Q = moment' * moment

        return Q[1]
    end

end

function msm_bootstrap(param::parameters, X::AbstractVector, Z::AbstractVector, U::AbstractVector, entrant::AbstractVector, B::Int64)
    est_id = Vector{Float64}(undef,1)
    est_num = Vector{Float64}(undef,1)
    est_id_rev = Vector{Float64}(undef,1)
    est_num_rev = Vector{Float64}(undef,1)
    b = 0
    while b < B
        Z_bt = Vector{Float64}[]
        for m in eachindex(entrant)
            temp = sample(Z[m], entrant[m]; replace = true, ordered = false)
            push!(Z_bt, temp)
        end

        entered_firm, decision = eq_firm_calc(tru_param, param, X, entrant, Z_bt, U)


        opt_identity = Optim.optimize(vars -> simulated_mm(vars, param, X, Z_bt, entered_firm, decision, entrant, 50, "identity"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
        estimates_identity = opt_identity.minimizer
        append!(est_id, estimates_identity)
        opt_number = Optim.optimize(vars -> simulated_mm(vars, param, X, Z_bt, entered_firm, decision, entrant, 50, "number"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
        estimates_msm = opt_number.minimizer
        append!(est_num, estimates_identity)
        opt_identity_rev = Optim.optimize(vars -> simulated_mm(vars, param, X, Z_bt, entered_firm, decision, entrant, 50, "identityrev"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
        estimates_identity_rev = opt_identity_rev.minimizer
        append!(est_id_rev, estimates_identity_rev)
        opt_number_rev = Optim.optimize(vars -> simulated_mm(vars, param, X, Z_bt, entered_firm, decision, entrant, 50, "numberrev"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
        estimates_number_rev = opt_number_rev.minimizer
        append!(est_num_rev, estimates_number_rev)



        b += 1
    end
    return (est_id[2:end], est_num[2:end], est_id_rev[2:end], est_num_rev[2:end])
end

msm_bootstrap (generic function with 1 method)

### (a-1) The correctly specified model: Using identities of firms and numbers of entered firms.

In [91]:
opt_identity = Optim.optimize(vars -> simulated_mm(vars, param, X, Z, entered_firm, decision, entrant, 200, "identity"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
estimates_identity = opt_identity.minimizer

3-element Vector{Float64}:
 1.9816565196623905
 0.851003693563963
 1.1334036592648404

### (a-2) The correctly specified model: Using just the numbers of entered firm.

In [103]:
opt_number = Optim.optimize(vars -> simulated_mm(vars, param, X, Z, entered_firm, decision, entrant, 200, "number"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
estimates_msm = opt_number.minimizer

3-element Vector{Float64}:
 1.933239484005392
 0.9789171418973519
 1.1882196697374585

### (b-1) The incorrectly specificed model: Using identities of firms and numbers of entered firms.

In [92]:
opt_identity_rev = Optim.optimize(vars -> simulated_mm(vars, param, X, Z, entered_firm, decision, entrant, 200, "identityrev"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
estimates_identity_rev = opt_identity_rev.minimizer

3-element Vector{Float64}:
 1.2231182031066525
 2.7818914310742255
 2.2687948757076404

### (b-2) The incorrectly specified model: Using just the numbers of entered firm.

In [105]:
opt_number_rev = Optim.optimize(vars -> simulated_mm(vars, param, X, Z, entered_firm, decision, entrant, 200, "numberrev"), ones(3), Optim.Options(show_trace = false, g_tol = 1e-5))
estimates_msm_rev = opt_number_rev.minimizer

3-element Vector{Float64}:
 1.9550171263455598
 1.917147396384817
 1.25705981799055

### Standard error: Bootstrap (Bootstrap simulation : 100 times)

In [83]:
ident, num, ident_rev, num_rev = msm_bootstrap(param, X, Z, u_firm_new, entrant, 100);

In [86]:
bt_1 = reshape(ident, 3, 100)
bt_2 = reshape(num, 3, 100)
bt_3 = reshape(ident_rev, 3, 100)
bt_4 = reshape(num_rev,3, 100)

μ_se =  sqrt(var(bt_1[1,:]))
σ_se = sqrt(var(bt_1[2,:]))
δ_se = sqrt(var(bt_1[3,:]))

μ_se_num = sqrt(var(bt_2[1,:]))
σ_se_num = sqrt(var(bt_2[2,:]))
δ_se_num = sqrt(var(bt_2[3,:]))

μ_se_rev =  sqrt(var(bt_3[1,:]))
σ_se_rev = sqrt(var(bt_3[2,:]))
δ_se_rev = sqrt(var(bt_3[3,:]))

μ_se_num_rev = sqrt(var(bt_4[1,:]))
σ_se_num_rev = sqrt(var(bt_4[2,:]))
δ_se_num_rev = sqrt(var(bt_4[3,:]));


### Case 1 (a-1): estimates ( $\mu$, $\sigma$, $\delta$)

In [94]:
estimates_identity

3-element Vector{Float64}:
 1.9816565196623905
 0.851003693563963
 1.1334036592648404

### Standard errors : $\mu$, $\sigma$, $\delta$

In [95]:
[μ_se, σ_se, δ_se]

3-element Vector{Float64}:
 0.11758075298612489
 0.10542881818444913
 0.14524510883341407

### Case 2 (a-2): estimates ( $\mu$, $\sigma$, $\delta$)


In [104]:
estimates_msm 

3-element Vector{Float64}:
 1.933239484005392
 0.9789171418973519
 1.1882196697374585

### Standard errors : $\mu$, $\sigma$, $\delta$


In [97]:
[μ_se_num, σ_se_num, δ_se_num]

3-element Vector{Float64}:
 0.11758075298612489
 0.10542881818444913
 0.14524510883341407

### Case 3 (b-1): estimates ( $\mu$, $\sigma$, $\delta$)



In [98]:
estimates_identity_rev

3-element Vector{Float64}:
 1.2231182031066525
 2.7818914310742255
 2.2687948757076404

### Standard errors : $\mu$, $\sigma$, $\delta$


In [99]:
[μ_se_rev, σ_se_rev, δ_se_rev]

3-element Vector{Float64}:
 0.3404853123415359
 0.23401493180375293
 0.4972343370078397

### Case 4 (b-2): estimates ( $\mu$, $\sigma$, $\delta$)



In [100]:
estimates_msm_rev

3-element Vector{Float64}:
 1.9550171263455598
 1.917147396384817
 1.25705981799055

### Standard errors : $\mu$, $\sigma$, $\delta$

In [101]:
[μ_se_num_rev, σ_se_num_rev,δ_se_num_rev]

3-element Vector{Float64}:
 0.14910509487251863
 0.13209731189491578
 0.2314522305717073

## Results: (Standard errrors are reported in the brackets)

|          	| Specification 1 (Identity & Number) 	| Specification 2 (Number) 	| Specification 3 (Identity & Number, Incorrect) 	| Specification 4 (Number, Incorrect) 	|
|:--------:	|:-----------------------------------:	|:------------------------:	|:----------------------------------------------:	|:-----------------------------------:	|
|   $\mu$  	|           1.9817 (0.1176)           	|      1.9332 (0.1176)     	|                 1.2231 (0.3405)                	|           1.9550 (0.1491)           	|
| $\sigma$ 	|           0.8510 (0.1054)           	|      0.9789 (0.1054)     	|                 2.7818 (0.2340)                	|           1.9171 (0.1320)           	|
| $\delta$ 	|           1.1334 (0.1452)           	|      1.1882 (0.1452)     	|                 2.2688 (0.4972)                	|           1.2570 (0.2315)           	|